# Turning Your Game to 11 with Unity Analytics and Data Science
---

This notebook accompanies a talk presented at Unite Melbourne 2017 by Marc Tanenbaum and Luke Dicken. The purpose is to highlight how, given a *Raw Data* export from Unity Analytics, you can use simple Data Science techniques in Python to gain powerful insights into your players.

In this example, we take some basic example data (about 1-week's worth of "level complete" events) to understand the types of players in a game. The data used is from a real game - live today - but has been anonymized.

It's also available as a raw Python script (rather than an iPython notebook).

In [1]:
import pandas as pd
from Utility.importer import importer
from sklearn.cluster import KMeans

## Import and setup

Here we'll use a custom import system to read the files and parse them into a more usable format. Because the data is from a live game, we won't share the raw data to Git, and will have a slightly different import approach (we'll start directly with the _rawcounter_ TSV file). We're grateful for the opportunity to present real data, but also want to respectful that it is proprietary.

In [2]:
# run the importer on the data folder
#df = importer('data') 
# send it to a TSV for Tableau visualisation
#df.to_csv('output-rawcounters.csv')

# Unzip this file and load from here
df = pd.DataFrame.from_csv('output-rawcounters.csv')
display(df)

,userid,duration_float,level_int,defeated_enemies_int,collected_gold_int,remaining_life_int
0,114ff8d2320a34c229dcad4e328d5751,104.02980,11,100,243,100
1,f91f8efc0bb2144ddb658afaa03aed1d,828.62750,11,13,134,100
2,34a2056271b994842b3d1984b95f96a0,165.41730,50,82,826,20
3,60af9eca30d994976b3bc32632ae0f03,1350.06900,10,100,547,80
4,3aea80c22078e4bc5905ae791207f81a,158.70450,2,100,458,100
5,a0803d375b9ea4b5783fae49936df021,391.85850,-1,100,347,40
6,f6921e441a5014c2aadc5f4c7ba2e039,289.25190,27,11,141,100
7,136597411bb07428eb9467fdaa8e9680,380.37390,19,97,1071,100
8,8d0aabe346b3d45269d4558252939f15,274.07960,12,90,253,20
9,a853d6700b7b7432cbb738d40eba8b0e,321.99340,45,100,690,80


## Reformulation

The dataset logs the level-complete counter - let's reshape it to be user centric
* There are 50 levels
* We know 4 things about each level: Life remaining, Gold collected, Enemies Defeated and Duration
* Users can repeat a level, so we need to decide how to aggregate - min, max, mean?

To start with we can group by userid and level that was completed, and for the sake of argument we'll aggregate by mean. Then we need to shift so that instead of having a row for every pair of (user, level) we have one long row for every row
* There are 50 possible levels, and 4 columns, so every row will have 201 columns (welcome to Big Data!)



In [3]:
# This groups by userid and level_int, using mean as the aggregate.
# Pandas defaults to making these groupings the indexes of the new table, so we reset that
udf = df.groupby(['userid', 'level_int']).aggregate('mean')
udf = udf.reset_index([0,1])

# This time we want just the userids, and we want to drop all the other data. We reset the index again as before
uids = df.groupby(['userid']).aggregate('mean')
uids = uids.drop(['duration_float','level_int','defeated_enemies_int','collected_gold_int','remaining_life_int'],1)
uids = uids.reset_index(0)


# Data Cleaning

Now _udf_ has the specific data we need, and _uids_ has a unique list of each userid
* Trick now is to step across the _udf_, select for the specific level (1-50) and join the data back to the _uid_ dataframe
* We need to ensure that nulls are recorded when we have no data - then later we'll replace those NaNs with the mean of the column
* Finally most clustering algorithms don't want to deal with labelled data (i.e. with the username) so let's also remove that since we don't care about who the individuals are anyhow

After we've cleaned everything up, we'll call the result _clusterData_

In [4]:
# initialise a new DF with all the userids
combined = uids
# for every level
for i in range(1,51):
    # select the subset of the data that's for that data
    subset = udf[(udf.level_int == i)]
    # we know which level this is so we can drop this in the subset
    subset = subset.drop('level_int',1)
    # but we do want to annotate our column names with the level number
    subset = subset.add_prefix(str(i)+"_")
    # except not the userid column, so set that one back
    subset = subset.rename(columns={str(i)+"_userid":"userid"})
    # this works like an SQL join - combined is joined to subset on userid.
    # we do an outer join to retain all the userids and put nulls (technically NaNs) is there's no data 
    combined = combined.merge(subset,on='userid',how='outer')

# Remove userIDs    
clusterData = combined.drop('userid',1)
# Replace NaN with column means
clusterData = clusterData.fillna(clusterData.mean())

display(clusterData)

,1_duration_float,1_defeated_enemies_int,1_collected_gold_int,1_remaining_life_int,2_duration_float,2_defeated_enemies_int,2_collected_gold_int,2_remaining_life_int,3_duration_float,3_defeated_enemies_int,...,48_collected_gold_int,48_remaining_life_int,49_duration_float,49_defeated_enemies_int,49_collected_gold_int,49_remaining_life_int,50_duration_float,50_defeated_enemies_int,50_collected_gold_int,50_remaining_life_int
0,1656.291000,100.000000,163.000000,100.000000,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,15834.820518,66.188330,718.715572,91.566022,407.160395,57.791118,466.847450,89.500000,617.500000,50.000000
1,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,15834.820518,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252
2,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,15834.820518,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252
3,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,15834.820518,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252
4,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,15834.820518,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252
5,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,15834.820518,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252
6,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,15834.820518,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252
7,132.512400,75.000000,155.000000,100.000000,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,15834.820518,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252
8,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,15834.820518,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252
9,110.196700,100.000000,175.000000,100.000000,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,15834.820518,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252


In [21]:
# Send to CSV to visualise user-level data in Tableau
clusterData.to_csv('output-userview.csv')


## Cluster-fun

We've now got a row per user with their average behaviour in the _clusterData_ DataFrame. It's time to do something clever with that!

We'll use the _K-means_ clustering algorithm that comes with SciKit Learn to figure out where the clusters in our data are. Because we have a lot of outliers, we'll aim to find a large number of clusters to address this, and because our clustering is fairly dense we'll use a random initialisation to encourage some separation.



In [5]:
# We turn the DF into a numPy matrix so KMeans can work with it
mat = clusterData.as_matrix()

# Now we can cluster
km = KMeans(n_clusters=20,init='random')
labels = km.fit_predict(mat)




## Understanding the Clustering

Now that the clustering algorithm has completed we can merge the cluster labels back into the dataset and visualise in Tableau again.

We've now clustered out users, and can take the centroids of those clusters and turn it into rules 

In [6]:
# TODO work out how to action the clustering algorithm
clustering = pd.Series(labels)
clusterData['cluster'] = clustering.values
clusterData.to_csv('output-clustered.csv')
# Send to CSV one last time to visualise again
display(clusterData)


,1_duration_float,1_defeated_enemies_int,1_collected_gold_int,1_remaining_life_int,2_duration_float,2_defeated_enemies_int,2_collected_gold_int,2_remaining_life_int,3_duration_float,3_defeated_enemies_int,...,48_remaining_life_int,49_duration_float,49_defeated_enemies_int,49_collected_gold_int,49_remaining_life_int,50_duration_float,50_defeated_enemies_int,50_collected_gold_int,50_remaining_life_int,cluster
0,1656.291000,100.000000,163.000000,100.000000,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,66.188330,718.715572,91.566022,407.160395,57.791118,466.847450,89.500000,617.500000,50.000000,2
1,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252,14
2,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252,9
3,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252,4
4,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252,4
5,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252,4
6,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252,14
7,132.512400,75.000000,155.000000,100.000000,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252,2
8,222.892573,84.451045,15860.660054,100.002907,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252,17
9,110.196700,100.000000,175.000000,100.000000,269.908999,89.405913,10118.521855,100.0,312.362031,90.565617,...,66.188330,718.715572,91.566022,407.160395,57.791118,654.301751,91.429989,584.652572,57.307252,2


## That's All Folks!

Seriously. Turns out this stuff isn't actually that scary huh?

Questions? Comments?

* Download this notebook: <https://github.com/LukeDicken/UniteMelbourneDemo2017>
* Contact: <ldicken@zynga.com>